In [1]:
# Instalamos pyspark y py4j
!pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=d3b685c8fcfbef42dcadf84ee74030bed4c7e23a059cbe78d53d19874333df4d
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

Spark = SparkSession.builder.appName("cjqm").getOrCreate()
SparkContext = Spark.sparkContext

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

# 1. Cree un RDD llamado lenguajes que contenga los siguientes lenguajes de programación: Python, R, C, Scala, Rugby y SQL.

In [ ]:
rdd_lenguaje = SparkContext.parallelize(['Python', 'R', 'C', 'Scala', 'Rugby', 'Sql'])
rdd_lenguaje.collect()

['Python', 'R', 'C', 'Scala', 'Rugby', 'Sql']

#  C) Cree un nuevo RDD que solo contenga aquellos lenguajes de          programación que comiencen con la letra R.

In [ ]:
rdd_filtrado_con_r = rdd_lenguaje.filter(lambda x: x.startswith('R'))
rdd_filtrado_con_r.collect()

['R', 'Rugby']

#2. Cree un RDD llamado pares que contenga los números pares existentes en el intervalo [20;30].

In [ ]:
rdd = SparkContext.parallelize([20,21,22,23,24,25,26,27,28,29,30,31])
rdd.collect()

[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

In [ ]:
rdd_pares = rdd.filter(lambda x: x % 2 == 0)
rdd_pares.collect()

[20, 22, 24, 26, 28, 30]

In [ ]:
numeros_pares = [x for x in range(20, 31) if x % 2 == 0]
pares_rdd = sc.parallelize(numeros_pares)
pares_rdd.collect()

[20, 22, 24, 26, 28, 30]

#  b) Obtenga una lista compuesta por los números pares en el intervalo [20;30] y sus respectivas raíces cuadradas.

In [ ]:
import math

# Crear una lista que contiene cada número par y su raíz cuadrada
pares_con_raiz = pares.flatMap(lambda x: [x, math.sqrt(x)])

# mostrar el contenido del nuevo RDD para verificar
pares_con_raiz.collect()


[20,
 4.47213595499958,
 22,
 4.69041575982343,
 24,
 4.898979485566356,
 26,
 5.0990195135927845,
 28,
 5.291502622129181,
 30,
 5.477225575051661]

# C) Eleve el número de particiones del RDD sqrt a 20.

In [ ]:
# Elevar el número de particiones del RDD 'pares_con_raiz' a 20
pares_con_raiz_20_particiones = pares_con_raiz.repartition(20)

# Verificar el número de particiones
pares_con_raiz_20_particiones.getNumPartitions()

20

#       D) Si tuviera que disminuir el número de particiones luego de haberlo  establecido en 20, ¿qué función utilizaría para hacer más eficiente su código?
# Respuesta: Utilizar la funcion Coalesce

In [ ]:
# Disminuir el número de particiones del RDD a 10
pares_con_raiz_10_particiones = pares_con_raiz_20_particiones.coalesce(12)

# Verificar el número de particiones
pares_con_raiz_10_particiones.getNumPartitions()

12

#3. Cree un RDD del tipo clave valor a partir de la lectura del txt de transacciones.
#txt:
#(1001, 52.3)
#(1005, 20.8)
#(1001, 10.1)
#(1004, 52.7)
#(1005, 20.7)
#(1002, 85.3)
#(1004, 20.9)
#Tenga en cuenta que deberá procesar el RDD leído para obtener el resultado solicitado. Supongamos
#que el RDD resultante de tipo clave valor refleja las transacciones realizadas por número de
#cuentas. Obtenga el monto total por cada cuenta (Suma).
#Tip: Cree su propia función para procesar el RDD leído.

In [ ]:
!touch transacciones.txt

!echo "(1001, 52.3)" >> transacciones.txt
!echo "(1005, 20.8)" >> transacciones.txt
!echo "(1001, 10.1)" >> transacciones.txt
!echo "(1004, 52.7)" >> transacciones.txt
!echo "(1005, 20.7)" >> transacciones.txt
!echo "(1002, 85.3)" >> transacciones.txt
!echo "(1004, 20.9)" >> transacciones.txt

In [ ]:
archivo_transacciones = "./transacciones.txt"
transacciones = sc.textFile(archivo_transacciones)
def procesar_linea(linea):
    # Eliminar los paréntesis y dividir por la coma
    cuenta, monto = linea.strip("()").split(", ")
    # Retornar una tupla (clave, valor)
    return (int(cuenta), float(monto))
transacciones_kv = transacciones.map(procesar_linea)
monto_total_por_cuenta = transacciones_kv.reduceByKey(lambda x, y: x + y)
resultado = monto_total_por_cuenta.collect()
for cuenta, total in resultado:
    print(f"Cuenta: {cuenta}, Monto total: {total:.2f}")